<a href="https://colab.research.google.com/github/bartoszrud/free-form-VQA/blob/main/T5_on_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/bartoszrud/free-form-VQA/image-captions/data/prediction_val_sub01/preditctions_gt_q.csv?token=GHSAT0AAAAAABWBBH737ATDHIC6JG2CKEWGYWMHBOQ')

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 6.6 MB 47.7 MB/s 
     |████████████████████████████████| 596 kB 71.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.spatial
import matplotlib.pyplot as plt

from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset
import torch
import pandas as pd
from pprint import pprint

import sys
import os
import glob

In [4]:
!pip install --upgrade simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.9 MB/s 
     |████████████████████████████████| 3.5 MB 53.2 MB/s 
     |████████████████████████████████| 527 kB 73.1 MB/s 
     |████████████████████████████████| 829 kB 75.3 MB/s 
     |████████████████████████████████| 419 kB 70.7 MB/s 
     |████████████████████████████████| 140 kB 70.7 MB/s 
     |████████████████████████████████| 952 kB 65.0 MB/s 
     |████████████████████████████████| 880 kB 62.5 MB/s 
     |████████████████████████████████| 1.1 MB 53.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 45.4 MB/s 
     |████████████████████████████████| 271 kB 62.7 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6873 sha256=4a2ba993404fd28d2ef16263ea71db27e3050fc89b9acef393187e8388194858
  Stored in directory: /root/.cache/pip/wheels/30/1b/96

In [5]:
q_list = "question: " + data['question']                          # questions list to feed the model
n_list = data['answer-gt'] + " </s>" # answers list to feed the model
#" " + data["answer-generated"] + 

dict_data = {'source_text': q_list,
      'target_text': n_list}

df = pd.DataFrame(dict_data)
df.head()

,source_text,target_text
0,question: How many cars are on the crosswalk,0 0 cars are on the crosswalk </s>
1,question: Are these people wearing helmets,No these people are not wearing helmets </s>
2,question: Are the men riding a bicycle,No the men are not riding a bicycle </s>
3,question: How many servings of pizza,8 8 servings of pizza </s>
4,question: Is the man barefoot,No the man is not barefoot </s>


In [6]:
from simplet5 import SimpleT5

Global seed set to 42


In [7]:
train_data, val_data = train_test_split(df[:-100], test_size=0.2)
test_data = df[-100:]
train_data.shape, val_data.shape, test_data.shape

((17783, 2), (4446, 2), (100, 2))

In [8]:
train_data = train_data.applymap(str)
val_data = val_data.applymap(str)

In [9]:
test_data = test_data.applymap(str)

In [10]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df = train_data,
            eval_df = val_data, 
            source_max_token_len=24, 
            target_max_token_len=34, 
            batch_size=8, max_epochs=3, use_gpu=True)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  def build_inputs_with_special_tokens(


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
model.load_model("t5","/content/outputs/simplet5-epoch-2-train-loss-0.3579-val-loss-0.3987")

In [12]:
test_data['paraphrased'] = test_data.apply(lambda row : model.predict(row['source_text']), axis=1)
test_data

,source_text,target_text,paraphrased
22229,question: Is the woman focused on her kitchen ...,No the woman is not focused on her kitchen wor...,[Yes the woman is focused on her kitchen work]
22230,question: What time is displayed on the clock ...,It is 2203 </s>,[It is 430]
22231,question: What color is the dress,The dress is gray </s>,[The dress is white]
22232,question: Is this person a cook,Yes this person is a cook </s>,[No this person is not a cook]
22233,question: What country is this taken in,Iran is this taken in </s>,[Usa is this taken in]
...,...,...,...
22324,question: What gender are the children,Male are the children </s>,[Male are the children]
22325,question: Is the guy in mid air,Yes the guy is in mid air </s>,[No the guy is not in mid air]
22326,question: What city is this,New york is this </s>,[London is this]
22327,question: How many bolts are holding the sign ...,4 4 bolts are holding the sign onto the buildi...,[0 0 bolts are holding the sign onto the build...


In [13]:
test_data.to_csv(r"/content/outputs/para_phrased.csv", index=False)